#   Bibliotecas

- Versões das bibliotecas no README

In [44]:
import pandas as pd
import random 
import numpy as np
import re
from scipy.sparse import hstack #utilizada para a junção de matrizes

In [45]:
#Pré processamento de colunas

from sklearn.model_selection import train_test_split #Divisão do dataset entre treino e teste
from sklearn.feature_extraction.text import TfidfVectorizer #vetorização do dataset
from sklearn.preprocessing import OneHotEncoder #aplicação de dummy

In [46]:
#Pré processamento de texto


import nltk
from nltk import word_tokenize #tokenizador
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords   #stopword

nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [47]:
#import de funções do script de pre_processamento

from pre_processamento import pre_processamento, text_preprocess, textcolumns_junct,stemming

In [48]:
# Modelo a ser testado
import tensorflow as tf
from keras import layers
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.utils import plot_model

In [49]:
#metricas

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error

In [50]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8972256609375542872
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2252026676
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10233264540272622807
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


#   Dados

In [51]:
train = pd.read_csv(r"C:\Users\angel\Desktop\bootcamp\train.tsv", sep='\t')
test = pd.read_csv(r"C:\Users\angel\Desktop\bootcamp\test_stg2.tsv", sep='\t')

In [52]:
train,test = pre_processamento(train,test)

#   Split

In [53]:
#Separação em treino e teste para começarmos as transformações, protegendo nossos dados de teste de possíveis vazamentos e das mudanças de hyperparametros. 

X1, Xtest = train_test_split(train, test_size=0.1, random_state=10) #Divisão em 90/10

In [54]:
#Separação em treino e teste para começarmos as transformaçõs, protegendo nossos dados de validação de possíveis vazamentos. 

Xtrain, Xval = train_test_split(X1, test_size=0.2, random_state=10) #Divisão em 80/20

In [55]:
print(Xtrain.shape) 
print(Xval.shape)
print(Xtest.shape)

(1066795, 13)
(266699, 13)
(148167, 13)


#   Pré-processamento 

In [56]:
#Função pra juntar as colunas de texto em uma só coluna

Xtrain = textcolumns_junct(Xtrain) 
Xval = textcolumns_junct(Xval) 
Xtest = textcolumns_junct(Xtest) 

In [57]:
#Convertendo tudo para string

Xtrain["item_description"] = Xtrain["item_description"].astype(str)
Xval["item_description"] = Xval["item_description"].astype(str)
Xtest["item_description"] = Xtest["item_description"].astype(str)

In [58]:
#Limpando texto com regex.
Xtrain["item_description"] = Xtrain["item_description"].apply(lambda x: text_preprocess(x))
Xval["item_description"] = Xval["item_description"].apply(lambda x: text_preprocess(x))
Xtest["item_description"] = Xtest["item_description"].apply(lambda x: text_preprocess(x))

In [59]:
#tokenizando

Xtrain["item_description"] = Xtrain["item_description"].apply(lambda x: word_tokenize(x))
Xval["item_description"] = Xval["item_description"].apply(lambda x: word_tokenize(x))
Xtest["item_description"] = Xtest["item_description"].apply(lambda x: word_tokenize(x))

In [60]:
Xtrain["item_description"] = Xtrain["item_description"].apply(lambda x: stemming(x))
Xval["item_description"] = Xval["item_description"].apply(lambda x: stemming(x))
Xtest["item_description"] = Xtest["item_description"].apply(lambda x: stemming(x))

In [61]:
Xtrain["item_description"]=[" ".join(review) for review in Xtrain["item_description"].values]
Xval["item_description"]=[" ".join(review) for review in Xval["item_description"].values]
Xtest["item_description"]=[" ".join(review) for review in Xtest["item_description"].values]

In [62]:
#Xtrain.to_pickle(r"C:\Users\angel\Desktop\bootcamp\Xtrain.pkl")
#Xval.to_pickle(r"C:\Users\angel\Desktop\bootcamp\Xval.pkl")
#Xtest.to_pickle(r"C:\Users\angel\Desktop\bootcamp\Xtest.pkl")

In [86]:
Xtrain = pd.read_pickle(r"C:\Users\angel\Desktop\bootcamp\Xtrain.pkl")
Xval = pd.read_pickle(r"C:\Users\angel\Desktop\bootcamp\Xval.pkl") 
Xtest = pd.read_pickle(r"C:\Users\angel\Desktop\bootcamp\Xtest.pkl") 

In [87]:
#treinando o modelo com valores até $250, que representam 89% do dataset
Xtrain = Xtrain.drop(Xtrain[~((Xtrain['price'] > 0) & (Xtrain['price'] <= 250))].index)

In [92]:
Xtrain.head()

,item_condition_id,category_name,shipping,item_description,stock,gen_cat,sub1_cat,sub2_cat,datetime_month,datetime_year
743320,1,Beauty/Makeup/Face,1,new face sweetheart blush bronzer face face bu...,11,Beauty,Makeup,Face,1.0,2018.0
313184,1,Vintage & Collectibles/Supplies/Fabric,1,bundl 6 red ish fabric clearanc size vari 1 ya...,15,Vintage & Collectibles,Supplies,Fabric,10.0,2018.0
1367929,1,Men/Men's Accessories/Hats,1,suprem camo facemask suprem brand new free shi...,4,Men,Men's Accessories,Hats,4.0,2018.0
41968,1,Beauty/Makeup/Makeup Sets,1,ipsi bag makeup brush sephora ipsi cosmet bag ...,19,Beauty,Makeup,Makeup Sets,11.0,2018.0
633793,1,Women/Swimwear/One-Piece,0,new acacia bronx one piec heliconia acacia swi...,17,Women,Swimwear,One-Piece,9.0,2018.0


In [89]:
ytrain = Xtrain["price"]
Xtrain = Xtrain.drop("price", axis=1)
Xtrain.shape


(1062337, 10)

In [90]:
yval= Xval["price"]
Xval = Xval.drop("price", axis = 1)
Xval.shape

(266699, 10)

In [91]:
ytest = Xtest["price"]
Xtest = Xtest.drop("price", axis = 1)
Xtest.shape

(148167, 10)

# Transformadores

# Rede Neural

In [93]:
Xtrain.columns

Index(['item_condition_id', 'category_name', 'shipping', 'item_description',
       'stock', 'gen_cat', 'sub1_cat', 'sub2_cat', 'datetime_month',
       'datetime_year'],
      dtype='object')

In [98]:
tf.keras.backend.clear_session()

#input para shipping
input_shipping = layers.Input(shape=)
hidden1_shipping = layers.Dense(512,activation="relu")(input_shipping)

#input para item_description
input_item_description = layers.Input(shape=)
hidden1_item_description = layers.Dense(512,activation="relu")(input_item_description)
drop1_item_description = layers.Dropout(0.2)(hidden1_item_description)
hidden2_item_description = layers.Dense(512,activation="relu")(drop1_item_description)

#input para gen_cat
input_gen_cat= layers.Input(shape=)
hidden1_gen_cat = layers.Dense(512,activation="relu")(input_gen_cat)

#input para sub1_cat
input_sub1_cat = layers.Input(shape=)
hidden1_sub1_cat = layers.Dense(512,activation="relu")(input_sub1_cat)

#input para sub2_cat
input_sub2_cat = layers.Input(shape=)
hidden1_sub2_cat = layers.Dense(512,activation="relu")(input_sub2_cat)

#Concatenando 
concat = layers.Concatenate()([hidden1_shipping, 
                             hidden2_item_description,
                             hidden1_gen_cat,
                             hidden1_sub1_cat,
                             hidden1_sub2_cat])

#Adicionando Layer à saida do concat

hidden1 = layers.Dense(512,activation="relu")(concat)
drop1 = layers.Dropout(0.2)(hidden1)
hidden2 = layers.Dense(512,activation="relu")(drop1)
drop2 = layers.Dropout(0.2)(hidden2)

#Saida
output = layers.Dense(1, activation="relu")(drop2)

#Modelo
model = Model(inputs = [input_shipping,
                        input_item_description,
                        input_gen_cat,
                        input_sub1_cat,
                        input_sub2_cat], 
                                          outputs = output)






In [106]:

tf.keras.utils.plot_model(model, to_file="my_model.png", show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [ ]:
model.compile(loss="mean_squared_error", 
              metrics=['mean_squared_error', RootMeanSquaredError()], 
              optimizer = 'Adam')

In [ ]:

history =  model.fit(Xtrain,
                     ytrain.values, 
                     batch_size=2048,
                     epochs=25, 
                     verbose=True)